Given that we have concluded our EDA on the weather dataset, we have concluded that the relevant factors are 'Tavg', 'DewPoint', 'PrecipTotal', 'StnPressure', 'ResultSpeed', 'ResultDir'. We will now combine these weather features with both our train and test dataset.

From our research on [Culex Lifecycles](https://www.cdc.gov/mosquitoes/about/life-cycles/culex.html) and a paper on how the [climate affects the West Nile Virus spread](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4342965/), we have found out that temperature, humidity and rain affects multiple parts of a mosquito's lifecycle. This ranges from 0-10 days from when an egg is laid. Whereas wind and air pressure affect adult mosquitos. We have concluded that temperature, dew point and precipitation will have to be rolled but wind and air pressure will not need to be.

## Modeling: Logistic Regression with feature selection

In [1]:
# Load in libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, mean_squared_error, plot_roc_curve, roc_auc_score

from imblearn.over_sampling import SMOTE

### Datasets preparation

In [2]:
# Read in datasets
train_final = pd.read_csv('./Datasets/train_final.csv')
test_final = pd.read_csv('./Datasets/test_final.csv')

In [3]:
# We then drop the features that have coefficient of 0 and redefine X train, X test
X_train = train_final.drop(columns = ['Address','Block',
'Street',
'AddressNumberAndStreet',
'Latitude',
'Longitude',
'Species',
'Trap',
'AddressAccuracy',
'tot_mosquitos','WnvPresent','month_year','year', 'month', 'day',
                                     'tarsalis', 'erraticus', 'unspecified',
                                     'T040', 'T200B', 'T234', 'T090A', 'T090C',
                                     'T090B', 'T218C', 'T128A', 'T218B', 'T218A',
                                    'T002A', 'T002B', 'T200A', 'T065A'  ])
y_train = train_final['WnvPresent']
X_test = test_final.drop(columns = ['Address',
'Block',
'Street',
'AddressNumberAndStreet',
'Latitude',
'Longitude',
'Species',
'Trap',
'AddressAccuracy','Id','month_year', 'year', 'month', 'day',
                                   'tarsalis', 'erraticus', 'unspecified',
                                   'T040', 'T200B', 'T234', 'T090A', 'T090C',
                                     'T090B', 'T218C', 'T128A', 'T218B', 'T218A',
                                    'T002A', 'T002B', 'T200A', 'T065A'])

### Feature selection with SelectKBest

In [4]:
# Apply SelectKBest class to extract top 30 features
bestfeatures = SelectKBest(score_func=chi2, k=30)
fit = bestfeatures.fit(X_train, y_train)

In [5]:
# Making score from SelectKBest and columns from X_train into a dataframe
bfscore = pd.DataFrame(fit.scores_)
bfcolumns = pd.DataFrame(X_train.columns)

# Merging the 2 dataframes into 1 and renaming the columns
featurescores = pd.concat([bfcolumns, bfscore], axis=1)
featurescores.columns = ['features', 'score']

# Check the newly created dataframe
featurescores

,features,score
0,pipiens/restuans,1.662472
1,restuans,52.679662
2,pipiens,60.734052
3,salinarius,4.652398
4,territans,12.107445
...,...,...
161,PrecipTotal_roll2,4.263751
162,PrecipTotal_roll3,2.446925
163,PrecipTotal_roll4,3.948339
164,PrecipTotal_roll5,2.474238


In [6]:
# Check features with a score of more than 5
topfeaturesdf = featurescores[featurescores['score'] > 5].sort_values(by='score', ascending = False)
topfeaturesdf

,features,score
7,8,163.266917
149,DewPoint_roll2,99.697911
150,DewPoint_roll3,86.189361
148,DewPoint,78.695334
5,6,72.845618
151,DewPoint_roll4,71.073139
2,pipiens,60.734052
152,DewPoint_roll5,58.147104
1,restuans,52.679662
153,DewPoint_roll6,52.140625


In [7]:
# Check features with a score of less than 5
dropfeaturesdf = featurescores[featurescores['score'] < 5].sort_values(by='score', ascending = False)
dropfeaturesdf

,features,score
139,T233,4.691662
3,salinarius,4.652398
138,T235,4.616777
69,T014,4.490383
68,T013,4.380554
...,...,...
51,T162,0.003770
39,T135,0.001901
132,T226,0.000301
145,StnPressure,0.000197


In [8]:
# Create a list of features with score of less than 10 to be dropped from dataset
dropfeatures = list(dropfeaturesdf.features)
dropfeatures

['T233',
 'salinarius',
 'T235',
 'T014',
 'T013',
 'T017',
 'PrecipTotal_roll2',
 'T230',
 'PrecipTotal_roll4',
 'T148',
 'T200',
 'PrecipTotal_roll6',
 'T018',
 'T043',
 'T015',
 'T011',
 'T096',
 'T049',
 'T102',
 'T103',
 'T903',
 'T062',
 'T145',
 'T099',
 'T074',
 'PrecipTotal_roll5',
 'T069',
 'T079',
 'PrecipTotal_roll3',
 'T228',
 'T063',
 'T005',
 'T209',
 'T153',
 'T082',
 'T212',
 'T152',
 'T048',
 'T009',
 'pipiens/restuans',
 'T080',
 'T146',
 'T231',
 'T028',
 'T222',
 'T129',
 'T047',
 'T008',
 'T070',
 'T088',
 'T061',
 'T016',
 'T025',
 'T092',
 'T007',
 'T161',
 'T224',
 'T150',
 'T019',
 'T100',
 'T045',
 'T219',
 'T141',
 'T206',
 'T027',
 'T073',
 'T075',
 'T071',
 'T218',
 'T034',
 'T050',
 'T154',
 'T094',
 'T221',
 'T051',
 'T031',
 'T144',
 'T044',
 'T149',
 'T060',
 'T157',
 'T001',
 'T083',
 'T114',
 'T004',
 'T155',
 'T107',
 'T072',
 'T054C',
 'T054',
 'T215',
 'T159',
 'T229',
 'T138',
 'T158',
 'T238',
 'T065',
 'T151',
 'T078',
 'PrecipTotal',
 'T077',


In [9]:
# Drop the features that have score of less than 5 and redefine X train, X test
X_train = X_train.drop(columns = dropfeatures)
X_test = X_test.drop(columns = dropfeatures)

In [10]:
# Scale columns that need to be scaled
# Instantiate StandardScaler
ss = StandardScaler()
# Define list of feature to be scaled
scale_list = [
    'ResultSpeed', 'DewPoint', 'DewPoint_roll2',
    'DewPoint_roll3', 'DewPoint_roll4', 'DewPoint_roll5', 'DewPoint_roll6',
    'Tavg', 'Tavg_roll2', 'Tavg_roll3', 'Tavg_roll4', 'Tavg_roll5',
    'Tavg_roll6'
]
# Fit and transform X_train
X_train[scale_list] = ss.fit_transform(X_train[scale_list])
# Transform X_test
X_test[scale_list] = ss.transform(X_test[scale_list])

# SMOTE
sm = SMOTE(sampling_strategy={1: 8000}, random_state=42)
Xsm_train, ysm_train = sm.fit_resample(X_train, y_train)

# Dataframes now are Xsm_train, ysm_train and X_test

### Raw logistic regression with no tuning

In [11]:
# Sample with a logistic regression
logregbf = LogisticRegression()
logregbf.fit(Xsm_train, ysm_train)

D:\Users\user\anaconda3\envs\dsi\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
# Prepare for kaggle
predict_probs = logregbf.predict_proba(X_test).tolist()

pred = []
for i in range(0, len(predict_probs)):
               pred.append(predict_probs[i][1])
        
sample_submission_logregbf = pd.DataFrame()
sample_submission_logregbf['Id'] = test_final['Id']
sample_submission_logregbf['WnvPresent'] = pred
sample_submission_logregbf.reset_index(drop=True, inplace=True)
sample_submission_logregbf.to_csv('sample_submission_logregbf.csv', index=False)

### Scoring

In [13]:
# Accuracy and roc score of model
print('logregbf accuracy:', logregbf.score(Xsm_train, ysm_train))
print("logregbf roc:", roc_auc_score(ysm_train,  [i[1] for i in logregbf.predict_proba(Xsm_train)]))

logregbf accuracy: 0.7737881508078994
logregbf roc: 0.8190287164234025


logregbf = LogisticRegression() **Kaggle score: 0.68448**

### GridSearch

In [14]:
# GridSearch with the following parameters tuning
# 'C': Inverse of regularization strength, smaller C= stronger regularization: [0.1, 1, 10]
# 'max_iter': Maximum number of iterations taken for the solvers to converge: [100, 1000, 10000]
# 'penalty': Regularize with l1(lasso) or l2(ridge): ['l1', 'l2']
# 'solver': 'liblinear'
grid = {
    'C': (0.1, 1, 10),
    'max_iter': [100, 1000, 10000],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}
logreg = LogisticRegression()
logreg_cv = GridSearchCV(logreg, grid, cv=5, scoring='roc_auc')
logreg_cv.fit(Xsm_train, ysm_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': (0.1, 1, 10), 'max_iter': [100, 1000, 10000],
                         'penalty': ['l1', 'l2'], 'solver': ['liblinear']},
             scoring='roc_auc')

In [15]:
# Check the params that give the best accuracy
print("logreg_cv best parameters: ",logreg_cv.best_params_)

logreg_cv best parameters:  {'C': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}


In [16]:
# Fitting model with the best params
logregbf2 = LogisticRegression(C= 0.1, max_iter= 100, penalty= 'l2', solver= 'liblinear')
logregbf2.fit(Xsm_train, ysm_train)

LogisticRegression(C=0.1, solver='liblinear')

In [17]:
# Prepare for kaggle
predict_probs2 = logregbf2.predict_proba(X_test).tolist()

pred2 = []
for i in range(0, len(predict_probs2)):
               pred2.append(predict_probs2[i][1])
        
sample_submission_logregbf2 = pd.DataFrame()
sample_submission_logregbf2['Id'] = test_final['Id']
sample_submission_logregbf2['WnvPresent'] = pred2
sample_submission_logregbf2.reset_index(drop=True, inplace=True)
sample_submission_logregbf2.to_csv('sample_submission_logregbf2.csv', index=False)

### Scoring

In [18]:
# Accuracy and roc score of model
print('logregbf2 accuracy:', logregbf2.score(Xsm_train, ysm_train))
print("logregbf2 roc:", roc_auc_score(ysm_train,  [i[1] for i in logregbf2.predict_proba(Xsm_train)]))

logregbf2 accuracy: 0.763387606017458
logregbf2 roc: 0.8148165705875139


logregbf2 = LogisticRegression(C= 0.1, max_iter= 100, penalty= 'l2', solver= 'liblinear') **Kaggle score: 0.69160**

In [19]:
# Preparing to create a dataframe with features and coefficients
x_coef = list(logregbf2.coef_)
Xsmcolumns = Xsm_train.columns
x_coef = x_coef[0]

In [20]:
# Creating a dataframe with features and coefficients from model
r_coef = pd.DataFrame({'Features': Xsmcolumns, 'Coefficient': x_coef})

In [21]:
# Sorting the data by coefficients in descending order
r_coef.sort_values(by = 'Coefficient', ascending = False)

,Features,Coefficient
26,Tavg_roll3,1.548419
22,DewPoint_roll5,1.147692
29,Tavg_roll6,0.584064
21,DewPoint_roll4,0.559545
19,DewPoint_roll2,0.321677
5,8,0.259240
24,Tavg,0.246933
14,T900,0.150959
23,DewPoint_roll6,0.107866
10,T143,-0.063957


### Summary & conclusion

|        Model        	|                       Features used                       	|      Scaling     	|                               Best params                              	| Accuracy 	|  Roc  	| Kaggle score 	|
|:-------------------:	|:---------------------------------------------------------:	|:----------------:	|:----------------------------------------------------------------------:	|:--------:	|:-----:	|--------------	|
| Logistic Regression 	| 30 features chosen using SelectKBest with chi2 as scoring 	| Standard & Smote 	| Tavg_roll3, DewPoint_roll5, Tavg_roll6, DewPoint_roll4, DewPoint_roll2 	|   0.763  	| 0.814 	| 0.69160      	|

We tried to improve the logistic regression score by using SelectKbest to further drop features but it turned out to perform worse most probably due to underfitting because of higher bias. The top 5 features being weather features do further strengthen our theory that weather is a strong predictor and rolling according to the Culex lifecycle is in the right direction.